In [87]:
# Imports
import pandas as pd
from datetime import datetime

import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

In [51]:
# Extract Date, Time, Temperature, SumRain into data array
atg_obs = pd.read_csv('LFEM_data-.csv', low_memory=False, skiprows=1)
atg_obs = atg_obs.drop(columns=[' Pseudo-Julian-Date', 'Relhum', 'Dewpoint', ])

data_arr = atg_obs.values

In [56]:
def convert_date_time(vector):
    date_str, time_str, value1, value2 = vector
    date_obj = datetime.strptime(date_str, "%Y-%m-%d")
    time_obj = datetime.strptime(time_str, '%H:%M')
    time_0_24 = time_obj.hour + time_obj.minute / 60
    day_of_year = date_obj.timetuple().tm_yday
    return [day_of_year, time_0_24, value1, value2]
processed_data = np.array([convert_date_time(data) for data in data_arr])

In [65]:
print(f'Data Shape: {data_arr.shape}')
print(f'Data Entry: [Date, Time, Temperature, SumRain]')

# Pre-Processed
print(data_arr[0])

# Post-Processed
print(processed_data[0])

Data Shape: (521176, 4)
Data Entry: [Date, Time, Temperature, SumRain]
['2023-01-01' '07:00' 46.2 0.0]
[ 1.   7.  46.2  0. ]


In [77]:
x_data = processed_data[:, :3]
y_data = processed_data[:, 3:]
print(x_data[0], y_data[0])

[ 1.   7.  46.2] [0.]


In [85]:
# Simple Model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(3,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])

In [86]:
# Train the Model:
model.fit(x_data, y_data, epochs=50)

Epoch 1/50
16287/16287 ━━━━━━━━━━━━━━━━━━━━ 5s 303us/step - loss: 4.9107 - mae: 1.3652
Epoch 2/50
16287/16287 ━━━━━━━━━━━━━━━━━━━━ 5s 300us/step - loss: 1.8141 - mae: 0.7862
Epoch 3/50
16287/16287 ━━━━━━━━━━━━━━━━━━━━ 5s 299us/step - loss: 0.8534 - mae: 0.5171
Epoch 4/50
16287/16287 ━━━━━━━━━━━━━━━━━━━━ 5s 297us/step - loss: 0.6803 - mae: 0.4614
Epoch 5/50
16287/16287 ━━━━━━━━━━━━━━━━━━━━ 5s 298us/step - loss: 0.7472 - mae: 0.4922
Epoch 6/50
16287/16287 ━━━━━━━━━━━━━━━━━━━━ 5s 299us/step - loss: 0.5884 - mae: 0.4280
Epoch 7/50
16287/16287 ━━━━━━━━━━━━━━━━━━━━ 5s 296us/step - loss: 0.5490 - mae: 0.4133
Epoch 8/50
16287/16287 ━━━━━━━━━━━━━━━━━━━━ 5s 300us/step - loss: 0.5518 - mae: 0.4114
Epoch 9/50
16287/16287 ━━━━━━━━━━━━━━━━━━━━ 5s 297us/step - loss: 0.5165 - mae: 0.4012
Epoch 10/50
16287/16287 ━━━━━━━━━━━━━━━━━━━━ 5s 297us/step - loss: 0.5075 - mae: 0.4027
Epoch 11/50
16287/16287 ━━━━━━━━━━━━━━━━━━━━ 5s 297us/step - loss: 0.5005 - mae: 0.4013
Epoch 12/50
16287/16287 ━━━━━━━━━━━━━━━━━

In [88]:
# Evaluation (Is it worth, the loss is quite high suggesting that either the model architecture is quite poor (likely) or 
# that we need to better preprocess our data such as rain (also likely))